Load necessary packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
import csv
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import metrics
import pandas as pd
import tensorflow as tf
import os

from keras.preprocessing import image
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization, Conv2D, Input, Dense, Dropout, MaxPool2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras import callbacks
from keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.layers.convolutional import ZeroPadding2D
from keras.optimizers import Adamax
from keras.applications import ResNet50

result = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

Using TensorFlow backend.


Download Training Data from Google Drive

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1wn1WdiMqfaBkHjLGgAGo7mgB2iCCpabk'})  #From shareable link
download.GetContentFile('fer2013.csv')
!ls

adc.json  fer2013.csv  sample_data


Load data-set and splite into training-validation and test dataset

In [3]:
def load_data():
	train_data_X, train_data_Y = [], []
	validation_data_X, validation_data_Y = [], []
	test_data_X, test_data_Y = [], []
	with open("fer2013.csv", 'r') as csvfile:
		datareader = csv.reader(csvfile, delimiter =',')
		headers = next(datareader)
		print(headers) 
		for row in datareader:
			emotion = int(row[0])
			usage = row[2]
			pixels = [int(p) for p in row[1].split()]
			if usage == "Training":
				train_data_X.append(pixels)
				train_data_Y.append(emotion)
			elif usage == "PrivateTest":
				validation_data_X.append(pixels)
				validation_data_Y.append(emotion)
			else:
				test_data_X.append(pixels)
				test_data_Y.append(emotion)
	train_data_X, train_data_Y = np.array(train_data_X)/255.0, np.array(train_data_Y)
	validation_data_X, validation_data_Y = np.array(validation_data_X)/255.0, np.array(validation_data_Y)
	test_data_X, test_data_Y = np.array(test_data_X)/255.0, np.array(test_data_Y)

  
	print("train_data ", train_data_X.shape)
	print("validation_data ", validation_data_X.shape)
	print("test_data ", test_data_X.shape)
	return train_data_X, train_data_Y, validation_data_X, validation_data_Y, test_data_X, test_data_Y

train_data_X, train_data_Y, validation_data_X, validation_data_Y, test_data_X, test_data_Y = load_data()

['emotion', 'pixels', 'Usage']
train_data  (28709, 2304)
validation_data  (3589, 2304)
test_data  (3589, 2304)


Support Vector Machine


In [0]:
def evaluate(model, X, Y):
  predicted_Y = model.predict(X)
  accuracy = accuracy_score(Y, predicted_Y)
  return accuracy

def svm():
  model = SVC(gamma='auto')
  model.fit(train_data_X, train_data_Y) # 
  SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
  validation_accuracy = evaluate(model, validation_data_X,  validation_data_Y)
  print( "  - validation accuracy = {0:.1f}".format(validation_accuracy*100))
  test_accuracy = evaluate(model, test_data_X, test_data_Y)
  print( "  - test accuracy = {0:.1f}".format(test_accuracy*100))
  
  
  y_pred = model.predict(test_data_X)
  for i in y_pred:
    print(i)
  print("Classification report for classifier %s:\n%s\n"
      % (model, metrics.classification_report(y_pred, test_data_Y)))
  return model

model = svm()

Support Vector Machine


  - validation accuracy = 36.8
  - test accuracy = 38.5  
  
Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):  
  
              precision    recall  f1-score   support

           0       0.03      0.20      0.05        59
           1       0.00      0.00      0.00         0
           2       0.08      0.33      0.14       126
           3       0.81      0.39      0.53      1875
           4       0.33      0.35      0.34       611
           5       0.45      0.54      0.49       340
           6       0.33      0.35      0.34       578

   micro avg       0.38      0.38      0.38      3589  
   macro avg       0.29      0.31      0.27      3589  
   weighted avg       0.58      0.38      0.44      3589